[![Open In Wolfram Cloud](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/PR/wolfram-badge.svg)](https://www.wolframcloud.com/obj/gvarnavi/Published/02X_discrete-physical-systems.nb)

# Discrete Physical Systems

We'll investigate how we can model physical systems and then apply those systems to generate visually appealing patterns and generative art!

We'll start by using CAs to model discrete physical systems.
These systems are discrete in both space (CA cells live on a fixed grid) and time (CA iterations advance in discrete 'time' steps).
We'll relax those assumptions going forward.

**Note:** This notebook follows from the [01X_elementary-cellular-automata-wl.ipynb](https://github.com/gvarnavi/generative-art-iap/blob/master/01.27-Thursday/01X_elementary-cellular-automata-wl.ipynb) notebook.

First, let's re-define our vectorized implementation of a 9-cell Moore neighborhood

In [ ]:
  Moore[func__, lat_] :=   
     MapThread[func, 
       Map[RotateRight[lat, #] &, 
         {{0, 0}, {1, 0}, {0, -1}, {-1, 0}, 
           {0, 1}, {1,  -1}, {-1, -1}, {-1, 1}, 
           {1, 1}}], 2];

## Majority Rule

The simplest totalistic 2D rule is the so-called majority rule.
Essentially each cell evolves according to what the majority of its Moore Neighborhood dictates.

In [ ]:
TableForm[Transpose@Table[{i, Floor[i/5]}, {i, 9}], 
 TableHeadings -> {{"Number of Alive Cells", "Outcome"}, None}]

- If four or less of its neighbors (self included) are "alive", the cell gets lonely and "dies"
- If five or more of its neighbors (self included) are "alive", the cell "lives".

Since each cell has a value of 0 or 1, we can simply divide the total by 5 and take the Floor.
This way if the total is 5 or more, the floor of a number $\in[1,2]$ will be 1 where as if it's less than 5, the floor of a number $\in[0,1]$ will be 0.

In [ ]:
rule[x_] := Floor[x/5]

We can see this in action for a single iteration:

In [ ]:
testConfig = RandomInteger[{0, 1}, {10, 10}];
{ArrayPlot[testConfig], 
 ArrayPlot[rule[#] & /@ Moore[Plus, testConfig], ImageSize -> Small]}

We wrap everything in a `NestList` function and display a movie of the result

In [ ]:
majorityRule[lat_, t_] :=
  Module[{rule},
  rule[x_] := Floor[x/5]; 
  Attributes[rule] = Listable;                  
  NestList[rule[Moore[Plus, #]] &, lat, t]]

In [ ]:
SeedRandom[1992];
initMajor = RandomInteger[1, {100, 100}];
frames = ArrayPlot /@ majorityRule[initMajor, 50];
Export["gifs/majority-rule-animation.gif", frames,ImageSize->300]

![majority rule](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/majority-rule-animation.gif "majority rule")

As we see - this very quickly reaches a stable equilibrium and no-longer evolves.
We'll make incrementally better and better approximations to this below

## Crude Phase Separation
We'll use a slight modification, by "flipping" the interfaces to make it more dynamic:  
- If four or less of its neighbors (self included) are "alive", the cell gets lonely and "dies"
  - Cell "lives" if exactly 4 neighbors are alive
- If five or more of its neighbors (self included) are "alive", the cell "lives".
  - Cell "dies" if exactly 5 neighbors are alive

In [ ]:
TableForm[
 Transpose@
  ReplacePart[
   Table[{i, Floor[i/5]}, {i, 9}], {{4, 2} -> 1, {5, 2} -> 0}], 
 TableHeadings -> {{"Number of Alive Neighbors", "Outcome"}, None}]

In [ ]:
spinodalCrude[lat_,t_] :=
  Module[{rule},
  rule[4] = 1; rule[5] = 0;
  rule[x_] := Floor[x/5]; 
  Attributes[rule] = Listable;                  
  NestList[rule[Moore[Plus, #]] &, lat, t]]

In [ ]:
SeedRandom[1992];
initSpCrude = RandomInteger[1, {100, 100}];
framesSpCrude = ArrayPlot /@ spinodalCrude[initSpCrude, 50];
Export["gifs/spinodal-crude-animation.gif", framesSpCrude,ImageSize->300]

![spinodal rule](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/spinodal-crude-animation.gif "spinodal crude")

Slightly better...
Still missing stability of segregated phases and matter conservation

## Adding Phase Stability
Let's think a little closer what the physics ought to be, in order to achiever a better result.

First, since this is a physical system (e.g. obeying pair interactions) it makes sense to weigh first nearest neighbor interactions more importantly than second nearest-neighbor interactions. Moreover, most material boundaries are "diffuse" in nature, so we want to avoid sharp transitions.
The hyperbolic tangent function can help with that.

In [ ]:
Plot[1.3 Tanh[x], {x, -2, 2}]

**Note:** we chose the pre-factor 1.3 since 1/Tanh[$\pm$1]$\approx$ $\pm$1.3 

In [ ]:
phaseOrderingNonConserved[lat_,d_, t_] := Module[{separate},
  separate[x_, n_, e_, s_, w_, ne_, se_, sw_, nw_] := 
   1.3 Tanh[x] + d ((n + e + s + w)/6 + (ne + se + sw + nw)/12 - x);
  NestList[Moore[separate, #] &, lat, t]]

In [ ]:
SeedRandom[1992];
initSpNonConserved = RandomReal[{-1,1}, {100, 100}];
framesSpNonConserved = ArrayPlot /@ phaseOrderingNonConserved[initSpNonConserved, 0.5,50];
Export["gifs/spinodal-non-conserved.gif", framesSpNonConserved,ImageSize->300]

![spinodal non-conserved](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/spinodal-non-conserved.gif "spinodal non-conserved")

Note how the Tanh dependence is very typical of cell dynamics simulations obeying the Cahn Hilliard equation!

## Phase-Ordering Conservation
Finally, we can ensure the two species conserve their phases by performing one additional Moore-neighborhood averaging.

In [ ]:
phaseOrderingConserved[lat_, d_,  t_] :=
  Module[{nnave, Itn},   
    nnave[x_, n_, e_, s_, w_, ne_, se_, sw_, 
    nw_] := (n + e + s + w)/6 + (ne + se + sw + nw)/12;
   
    Itn[mat_] := d (Moore[nnave, mat] - mat) + 1.3 Tanh[mat] - mat;
  
    NestList[Function[y, (# + y - Moore[nnave, y])][Itn[#]] &, lat, 
   t]]

In [ ]:
SeedRandom[1992];
initSpRealistic = RandomReal[{-0.1,0.1}, {100, 100}];
framesSpRealistic = ArrayPlot /@ phaseOrderingConserved[initSpRealistic, 0.5,50];
Export["gifs/spinodal-conserved.gif", framesSpRealistic,ImageSize->300]

![spinodal conserved](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/spinodal-conserved.gif "spinodal conserved")

## One small step for code..
We'll use the Crude Spinodal Rule to illustrate the ease in which the aforementioned procedure can be generalized in three dimensions:

In [ ]:
steps = Tuples[Range[-1, 1], 3];
upToNNNN = SortBy[steps, Norm[#] &]

In [ ]:
majorityRule3D[lattice_, t_] := 
 Module[{Moore, rule}, 
  Moore[func__, lat_] := 
   MapThread[func, Map[RotateRight[lat, #] &, upToNNNN], 2];
   
  rule[13] = 1; rule[14] = 0;
  rule[x_] := Floor[x/14];
  
  Attributes[rule] = Listable;
  NestList[rule[Moore[Plus, #]] &, lattice, t]]

In [ ]:
SeedRandom[1992];
init3D = RandomInteger[{0,1}, {40,40,40}];
frames3D = Image3D /@ majorityRule3D[init3D,20];
Export["gifs/majority-3D.gif", frames3D,ImageSize->300]

![majority 3D](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/majority-3D.gif "majority 3D")

## Seeding with images!
Let's use our 2D phase-conserving spinodal evolution CA, but seed the evolution using images.  
What we're thinking here is starting off with a patern we'd like to evolve, and computationally adding 'oil' in the dark regions, and 'water' in the bright regions. At high temperatures, these mix and we can't really discern our image. As we quench the system (evolve our spinodal decomposition CA), the 'oil' and 'water' will start phase separating!

We'll start with a 'mystery' image and evolve it. Then, we'll show how to do this with your images.  
We import the initial seed from the cloud.

In [ ]:
ArrayPlot[
mystery=CloudGet[
"https://www.wolframcloud.com/obj/gvarnavi/02_discrete_physical_systems_mystery-image"
],ImageSize->200]

And then evolve it as usual:

In [ ]:
framesMystery = ArrayPlot /@ phaseOrderingConserved[mystery, 0.4,50];
Export["gifs/mystery-image-spinodal.gif", framesMystery,ImageSize->300]

![mystery image](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/mystery-image-spinodal.gif "mystery image spinodal")

## Starting from scratch
Let's do a similar exercise, starting from an image off the web, so you can try on images you like.  
Per popular request, we'll use an image of Princess Leia!

In [ ]:
princessLeia = Import["https://i.pinimg.com/736x/f3/9b/dd/f39bdd987b04d7ba95abb927b8e004be.jpg"];
resizedLeia = Blur[
  ImageCrop[
   ImageCrop[
    ImageResize[princessLeia, 220], {200, 280}, Top], {200, 200}, 
   Bottom], 3]

We might as-well do this on a color image, by operating on each RGB channel separately.  
We rescale the values from each channel to lie b/w -0.1 and 0.1

In [ ]:
channelSeparatedLeia = ColorSeparate[resizedLeia];
initializedLeia = Rescale[ImageData[#], {0, 1}, {-0.1, 0.1}] & /@ channelSeparatedLeia;

### Non-phase conserving
Let's start with our non-phase conserving CA, for 100 iterations.  
(Note: this will likely take a while to run)

In [ ]:
framesNonConserving["red"] = 
  phaseOrderingNonConserved[initializedLeia[[1]], 0.5, 100];
framesNonConserving["green"] = 
  phaseOrderingNonConserved[initializedLeia[[2]], 0.5, 100];
framesNonConserving["blue"] = 
  phaseOrderingNonConserved[initializedLeia[[3]], 0.5, 100];
  
framesNonConserving["combined"]=Table[
  ColorCombine[
   Table[ImageAdjust@Image[framesNonConserving[c][[frame]]], {c, {"red","green","blue"}}], 
   "RGB"],{frame,1,100,1}];
   
Export["gifs/leia-non-conserved.gif", framesNonConserving["combined"],ImageSize->300]

![non-conserved Leia](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/leia-non-conserved.gif "non-conserved Leia")

### Phase conserving
While this certainly looks cool - it clearly does not conserve the amount of each channel phase. Let's see if we can do better using our phase-conserving CA, for 250 iterations.  
(Note: this will take even longer to run)

In [ ]:
framesConserving["red"] = 
  phaseOrderingConserved[initializedLeia[[1]], 0.5, 250];
framesConserving["green"] = 
  phaseOrderingConserved[initializedLeia[[2]], 0.5, 250];
framesConserving["blue"] = 
  phaseOrderingConserved[initializedLeia[[3]], 0.5, 250];
  
framesConserving["combined"]=Table[
  ColorCombine[
   Table[ImageAdjust@Image[framesConserving[c][[frame]]], {c, {"red","green","blue"}}], 
   "RGB"],{frame,1,250,1}];
   
Export["gifs/leia-conserved.gif", framesConserving["combined"],ImageSize->300]

![conserved Leia](https://raw.githubusercontent.com/gvarnavi/generative-art-iap/master/01.27-Thursday/gifs/leia-conserved.gif "conserved Leia")